In [1]:
# install dependencies
!pip install youtube_dl
!pip install transformers
!pip install spacy
!python -m spacy download en_core_web_lg
!pip install sentencepiece
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [2]:
# Shout out to Nabil bd !
# https://www.github.com/Nabilphysics
# https://www.nabilbd.com

import youtube_dl
import os
import whisper
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline
import pandas as pd
import gc
# Credit: https://www.activestate.com/blog/how-to-do-text-summarization-with-python/
#punctuation = punctuation + '\n'
#punctuation

In [3]:
#define all functions

# Function to convert youtube link to embed link
# Credit: chatGPT (although chatGPT first mislead me)
def convert_to_embed(link):
    video_id = link.split("/")[-1]
    return 'https://www.youtube.com/embed/' + video_id

def download_ytvid_as_mp3():
    #video_url = input("enter url of youtube video:")
    #https://www.youtube.com/watch?v=6KUn7AKmC0Y
    video_info = youtube_dl.YoutubeDL().extract_info(url = youtube_link,download=False)
    # Your file location for Ubuntu
    myfile = "youtubeVideo.mp3"
   
    # If file exists, delete it.
    if os.path.isfile(myfile):
     os.remove(myfile)
    else:
     # If it fails, inform the user.
     print("Error: %s file not found" % myfile)
    
    #filename = f"{video_info['title']}.mp3"
    filename = f"{'youtubeVideo'}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    print("Download complete... {}".format(filename))


def summarize(text, per):
    nlp = spacy.load('en_core_web_lg')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary 

In [4]:
# load openAI model whisper
model = whisper.load_model("medium")
# Pick model
model_name = "google/pegasus-xsum"
# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
# Define summarization pipeline 
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)


100%|██████████████████████████████████████| 1.42G/1.42G [00:04<00:00, 365MiB/s]


In [5]:
df = pd.read_csv('/content/drive/MyDrive/ChatGPTvHumans/newdataset.csv')
df.head()

,Link,Title,Transcript,Summary
0,https://www.youtube.com/watch?v=2-VTLiy0Lls,Can you steal the most powerful wand in the wi...,NaN,NaN
1,https://www.youtube.com/watch?v=KO6bbZOc2B4,How to deal with rejection,NaN,NaN
2,https://www.youtube.com/watch?v=AESkozhaT4s,When are you actually an adult? - Shannon Odell,NaN,NaN
3,https://www.youtube.com/watch?v=BGxYMTftuzw,Mao Zedong's infamous mango cult - Vivian Jiang,NaN,NaN
4,https://www.youtube.com/watch?v=xqaX2yi0VF8,The bizarre world of parasitic wasps - Miles Z...,NaN,NaN


In [ ]:
for index in range(79,len(df)):
  
  try:
    os.remove('youtubeVideo.mp3')
  except:
    print("")
  try:  
    # read link from dataframe
    youtube_link = df.loc[index,['Link']].Link
    print("\nLink ",index,"/",len(df)," : " , youtube_link)

    # embed link
    print("\nEmbedding...")
    embed_link = convert_to_embed(youtube_link)
    print(embed_link)

    # download video
    download_ytvid_as_mp3()

    print("\nTranscribing...")
    result = model.transcribe('youtubeVideo.mp3')
    text = result["text"]
    print("Transcription:",text)

    res = len(text.split())
    # total no of words
    print ("\nThe number of words in string are : " + str(res))

    df.loc[index,['Transcript']] = text

    df.to_csv('/content/drive/MyDrive/ChatGPTvHumans/Copy of newdataset.csv')

  except:
    print("Skipped",index)
  
  # garbage collector
  # del result
  # del text
  gc.collect()




Link  79 / 666  :  https://youtu.be/E1KkQrFEl2I

Embedding...
https://www.youtube.com/embed/E1KkQrFEl2I
[youtube] E1KkQrFEl2I: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] E1KkQrFEl2I: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download]  64.3% of 10.75MiB at 79.40KiB/s ETA 00:49Skipped 79


Link  80 / 666  :  https://youtu.be/HjHMoNGqQTI

Embedding...
https://www.youtube.com/embed/HjHMoNGqQTI
[youtube] HjHMoNGqQTI: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] HjHMoNGqQTI: Downloading webpage
[download] Resuming download at byte 7246452
[download] Destination: youtubeVideo.mp3
[download]  30.8% of 23.19MiB at 56.26KiB/s ETA 04:51Skipped 80


Link  81 / 666  :  https://youtu.be/v3y8AIEX_dU

Embedding...
https://www.youtube.com/embed/v3y8AIEX_dU
[youtube] v3y8AIEX_dU: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] v3y8AIEX_dU: Downloading webpage
[download] Resuming download at byte 7498272
[downlo

ERROR: unable to download video data: HTTP Error 403: Forbidden


Skipped 84


Link  85 / 666  :  https://youtu.be/fXb02MQ78yQ

Embedding...
https://www.youtube.com/embed/fXb02MQ78yQ
[youtube] fXb02MQ78yQ: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] fXb02MQ78yQ: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 11.76MiB in 03:08
Download complete... youtubeVideo.mp3

Transcribing...
Transcription:  The Earth is a gigantic ball of semi-molten rock, with a heart of iron as hot as the surface of the Sun. Titanic amounts of heat left over from its birth and the radioactive decay of trillions of tons of radioactive elements find no escape but up. Currents of rock spanning thousands of kilometers carry this energy to the surface. Earth's crust is the only thing in their way. It feels solid to us, but it's only a fragile barrier, an apple skin around a flaming behemoth. True apocalypses can break through and unleash eruptions tens of times more powerful than all of our nuclear weapons combined, subjecti

ERROR: Sign in to confirm your age
This video may be inappropriate for some users.


Skipped 139


Link  140 / 666  :  https://youtu.be/XWYhJ7NNF1c

Embedding...
https://www.youtube.com/embed/XWYhJ7NNF1c
[youtube] XWYhJ7NNF1c: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] XWYhJ7NNF1c: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 5.79MiB in 02:06
[ffmpeg] Correcting container in "youtubeVideo.mp3"
Download complete... youtubeVideo.mp3

Transcribing...
Transcription:  Russia's propaganda machine is moving into high gear as the war in Ukraine nears its first anniversary. Vladimir Putin's shameless supporters in state media are making a mockery of Western weapons, but not everyone's buying it, CNN's Nick Robertson reports. Toothless cats battered Abrams, he says. The Russian state media anchor points to a huge graphic alleging flaws in the NATO tanks. His purposeful propaganda, playing down risks the Russian troops in Ukraine, does seem to be working. Sending tanks is going to be bad for the Americans and Germans, s

...

Skipped 289


Link  290 / 666  :  https://youtu.be/OPdkeoxNCnA

Embedding...
https://www.youtube.com/embed/OPdkeoxNCnA
[youtube] OPdkeoxNCnA: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] OPdkeoxNCnA: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 9.95MiB in 02:55
[ffmpeg] Correcting container in "youtubeVideo.mp3"
Download complete... youtubeVideo.mp3

Transcribing...
Transcription:  Before we get to AI on topic of revolutionary ideas, let me ask on a Reddit AMA. You said that is the earth flat is one of the favorite questions you've ever answered. Yeah. Speaking of revolutionary ideas. So your video on that people should definitely watch is really fascinating. Can you elaborate why you enjoyed answering this question so much? Yeah, well, it's a long story. I remember a long time ago, I was living in New York at the time, so it had to have been like 2009 or something. I visited the Flat Earth forums and this was before the Flat E

ERROR: unable to download video data: HTTP Error 403: Forbidden


Skipped 307


Link  308 / 666  :  https://youtu.be/K-Fc08X56R0

Embedding...
https://www.youtube.com/embed/K-Fc08X56R0
[youtube] K-Fc08X56R0: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] K-Fc08X56R0: Downloading webpage


ERROR: unable to download video data: HTTP Error 403: Forbidden


Skipped 308


Link  309 / 666  :  https://youtu.be/kTXTPe3wahc

Embedding...
https://www.youtube.com/embed/kTXTPe3wahc
[youtube] kTXTPe3wahc: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] kTXTPe3wahc: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 18.51MiB in 05:05
[ffmpeg] Correcting container in "youtubeVideo.mp3"
Download complete... youtubeVideo.mp3

Transcribing...
Transcription:  A portion of this video was sponsored by Norton 360. Classical mechanics is great. If you know the state of a system, say the position and velocity of a particle, then you can use an equation, Newton's second law, to calculate what that particle will do in the future. In quantum mechanics, if you know the quantum state of a particle, that is its wave function, you can use the Schrödinger equation to calculate what that particle will do in the future. Usually, it spreads out over time as it is doing here. Note, to make this animation, we really solve

In [ ]:
from google.colab import drive
drive.mount('/content/drive')